# **Piper vs. Cessna Notebook**

## Objectives

*   Answer business requirement 3: 
    * The client is interested in evaluating the “strength and weakness”-profile for two of their main competitors Piper and Cessna by evaluating the differences in performance for different features.

## Inputs

* outputs/datasets/collection/airplane_performance_study.csv

## Outputs

* generate code that answers business requirement 3 and can be used to build the Streamlit App "Regression Playground"-page






---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Imports

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import plotly.express as px

# Load Data

* Dropping the columns with Meta Data 'Model' and 'Company' since these are identifier variables not needed for the study
* Dropping the two Engine "size" features 'THR' (Force with unit lbf) and 'SHP' (Power with unit SHP). The "THR" column is occupied (otherwise showing NaN) when the "Engine Type" is categorized with Jet and the same goes for the "SHP" when the "Engine Type" is categorized with piston or propjet. These two features () are interesting from an Aircraft Design perspective but since these are different quantities with different units they become difficult/awkward to compare with each other.

In [ ]:
df = pd.read_csv("/workspace/data-driven-design/outputs/datasets/collection/airplane_performance_study.csv")
df.head(10)


---

# Data Exploration from the domain specific perspective of Airplane Design and Performance

---

## Piper vs. Cessna

We decided to pitch the Pier Aircraft against Cessna ...

Already Kaggle where we retrieved the data we could glean info about the data without lifting a finger. We see that the data (that will be transformed into categorical data) currently is unbalanced.

We have choosen here to pit two of the largest general aviation manufacturers Cessna with 24% of the data points and Piper standing for 13% of the datapoints against each other. These are also the two companies with the largest part of the data set. We have choosen a selection of key performance features and plotted them against Wing Span and the All Upp Weight (AUW).

Piper aircrafts appear to have their airplanes more focust in both Wing span and weight than Cessna so we only compare these intervalls where both companies are represented with data points. Perhaps we should cut of the intervall even before the linear regression takes place!

Interestingly enough it appears that the Pipers linear regression line have a pivot point around the middles so that they are better than Cessnas but worse than Cessna for the smaller wingspans and this pattern seems to be consistent across all the plotted performance except for perhaps Hmax. Interestingly enogh the opposite is true, higher weights makes Cessna win over Piper but lower weights makes Piper win over Cessna.

I think this is however becasue of the skewed data given becasue Cessnas regression line is influensed by the data outside of Pipers data range and if the data is not completely linear then logically that would have a large effect on the gradient of the regression line!

Is the data linear? Aerodynamic Scale factors (linked to RE-nr) would suggest that Performance would increase more than a linear increase as Wingspan increases.
Structural Scale effects would rather suggest the opposite, who wins!?

Observation gives that the performance parameters appear to be relativly linear for Wing Span however less so for Weight where the data is getting more flat as weight increases.

Despite having domain specific knowledge about Aircraft performance I do not see any obvious reason for this data-pattern!

<img src="/workspace/data-driven-design/images_notebook/cessna_vs_piper_kaggle.png" alt="Screenshot showing distribution between Cessna and Piper" height="200" />

In [ ]:
filtered_df = df[df['Company'].isin(['Cessna Aircraft Company', 'Piper Aircraft'])]
print(filtered_df)

In [ ]:
# Filter regression plots
plt.figure(figsize=(10, 6))

# Plot using Seaborn's regression plot
sns.regplot(data=filtered_df[filtered_df['Company'] == 'Cessna Aircraft Company'],
            x='Wing_Span', y='Range',
            label='Cessna Aircraft Company', color='blue', ci=None)

sns.regplot(data=filtered_df[filtered_df['Company'] == 'Piper Aircraft'],
            x='Wing_Span', y='Range',
            label='Piper Aircraft', color='orange', ci=None)

plt.xlim(filtered_df['Wing_Span'].min() - 5, filtered_df['Wing_Span'].max() + 5)
plt.ylim(filtered_df['Range'].min() - 5, filtered_df['Range'].max() + 5)

# Set limits for x and y axes
plt.title('Range vs. Wingspan for Cessna and Piper Aircraft')
plt.xlabel('Wing Span (ft)')
plt.ylabel('Range(N.M.)')
plt.legend()
plt.grid()
plt.show()

# Compare regression coefficients
for company in ['Cessna Aircraft Company', 'Piper Aircraft']:
    X = filtered_df[filtered_df['Company'] == company]['Wing_Span'].values.reshape(-1, 1)
    y = filtered_df[filtered_df['Company'] == company]['Range'].values
    model = LinearRegression().fit(X, y)
    print(f"{company} - Coefficient: {model.coef_[0]:.2f}, Intercept: {model.intercept_:.2f}")


In [ ]:
# Parameters to plot
parameters = ['Vmax', 'Vcruise', 'Range', 'Hmax', 'ROC']

# Plotting for Wingspan
plt.figure(figsize=(15, 10))
for i, param in enumerate(parameters, start=1):
    plt.subplot(3, 2, i)

    # Create a scatter plot with regression lines for Cessna
    sns.regplot(data=filtered_df[filtered_df['Company'] == 'Cessna Aircraft Company'],
                x='Wing_Span', y=param,
                label='Cessna Aircraft Company', color='blue', ci=None)

    # Create a scatter plot with regression lines for Piper
    sns.regplot(data=filtered_df[filtered_df['Company'] == 'Piper Aircraft'],
                x='Wing_Span', y=param,
                label='Piper Aircraft', color='orange', ci=None)

    # Set limits
    plt.xlim(filtered_df['Wing_Span'].min() - 5, filtered_df['Wing_Span'].max() + 5)
    plt.ylim(filtered_df[param].min() - 5, filtered_df[param].max() + 5)

    # Add titles and labels
    plt.title(f'{param} vs. Wing_Span for Cessna Aircraft Company and Piper Aircraft')
    plt.xlabel('Wing_Span (ft)')
    plt.ylabel(param)
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.show()

# Plotting for AUW
plt.figure(figsize=(15, 10))
for i, param in enumerate(parameters, start=1):
    plt.subplot(3, 2, i)

    # Create a scatter plot with regression lines for Cessna
    sns.regplot(data=filtered_df[filtered_df['Company'] == 'Cessna Aircraft Company'],
                x='AUW', y=param,
                label='Cessna', color='blue', ci=None)

    # Create a scatter plot with regression lines for Piper
    sns.regplot(data=filtered_df[filtered_df['Company'] == 'Piper Aircraft'],
                x='AUW', y=param,
                label='Piper', color='orange', ci=None)

    # Set limits
    plt.xlim(filtered_df['AUW'].min() - 100, filtered_df['AUW'].max() + 100)
    plt.ylim(filtered_df[param].min() - 5, filtered_df[param].max() + 5)

    # Add titles and labels
    plt.title(f'{param} vs. AUW for Cessna Aircraft Company and Piper Aircraft')
    plt.xlabel('AUW (lbs)')
    plt.ylabel(param)
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.show()

# Compare regression coefficients for each parameter
for param in parameters:
    for company in ['Cessna Aircraft Company', 'Piper Aircraft']:
        X_wing_Span = filtered_df[filtered_df['Company'] == company]['Wing_Span'].values.reshape(-1, 1)
        y_param = filtered_df[filtered_df['Company'] == company][param].values
        model_wing_Span = LinearRegression().fit(X_wing_Span, y_param)
        print(f"{company} ({param} vs. Wing_Span) - Coefficient: {model_wing_Span.coef_[0]:.2f}, Intercept: {model_wing_Span.intercept_:.2f}")

        X_auw = filtered_df[filtered_df['Company'] == company]['AUW'].values.reshape(-1, 1)
        model_auw = LinearRegression().fit(X_auw, y_param)
        print(f"{company} ({param} vs. AUW) - Coefficient: {model_auw.coef_[0]:.2f}, Intercept: {model_auw.intercept_:.2f}")


Now with also trimmed axis
We have here trimmed the axis so that Cessnas regression line is based only on the data points that are within the Pipers data range (for the wingspan and Weight). Only than can we have a fair comparison. 

It appears from this analysis that Piper is the winner however we have here not consider the following factors:
* Perhaps the Piper aircrafs in this analysed data range are newer (first flight of more recent years) than the Cessna airplanes in the data
* The Merit of an airplane is not measured by only one aspects, a few number of perforamnce features (albeit key such) but a complete set of features including: Practicality, strenght, price, longervity etc.

In [ ]:
# Parameters to plot
parameters = ['Vmax', 'Vcruise', 'Range', 'Hmax', 'ROC']


# Function to trim Cessna data based on Piper data ranges
def trim_cessna_data(piper_data, cessna_data, param):
    """
    Trim Cessna data to include only rows where the specified parameter
    falls within the range of that parameter in Piper data.

    Parameters:
    piper_data (DataFrame): DataFrame containing Piper data.
    cessna_data (DataFrame): DataFrame containing Cessna data.
    param (str): The parameter column name to compare.

    Returns:
    DataFrame: Filtered Cessna data within the Piper parameter range.
    """
    piper_min = piper_data[param].min()
    piper_max = piper_data[param].max()
    return cessna_data[(cessna_data[param] >= piper_min) & (cessna_data[param] <= piper_max)]

# Plotting for Wingspan
plt.figure(figsize=(15, 10))
for i, param in enumerate(parameters, start=1):
    plt.subplot(3, 2, i)

    # Get Piper data for the parameter
    piper_data = filtered_df[filtered_df['Company'] == 'Piper Aircraft']

    # Trim Cessna data for fair comparison
    cessna_data_trimmed = trim_cessna_data(piper_data,
                                           filtered_df[filtered_df['Company'] == 'Cessna Aircraft Company'],
                                           param)

    # Create a scatter plot with regression lines for trimmed Cessna
    sns.regplot(data=cessna_data_trimmed,
                x='Wing_Span', y=param,
                label='Cessna Aircraft Company (Trimmed)', color='blue', ci=None)

    # Create a scatter plot with regression lines for Piper
    sns.regplot(data=piper_data,
                x='Wing_Span', y=param,
                label='Piper Aircraft', color='orange', ci=None)

    # Set limits based on trimmed data
    plt.xlim(cessna_data_trimmed['Wing_Span'].min() - 5, cessna_data_trimmed['Wing_Span'].max() + 5)
    plt.ylim(filtered_df[param].min() - 5, filtered_df[param].max() + 5)

    # Add titles and labels
    plt.title(f'{param} vs. Wing_Span for Cessna and Piper Aircraft')
    plt.xlabel('Wing_Span (ft)')
    plt.ylabel(param)
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.show()

# Plotting for AUW
plt.figure(figsize=(15, 10))
for i, param in enumerate(parameters, start=1):
    plt.subplot(3, 2, i)

    # Trim Cessna data for fair comparison based on Piper data
    cessna_data_trimmed_auw = trim_cessna_data(piper_data,
                                               filtered_df[filtered_df['Company'] == 'Cessna Aircraft Company'],
                                               param)

    # Create a scatter plot with regression lines for trimmed Cessna
    sns.regplot(data=cessna_data_trimmed_auw,
                x='AUW', y=param,
                label='Cessna (Trimmed)', color='blue', ci=None)

    # Create a scatter plot with regression lines for Piper
    sns.regplot(data=piper_data,
                x='AUW', y=param,
                label='Piper', color='orange', ci=None)

    # Set limits based on trimmed data
    plt.xlim(cessna_data_trimmed_auw['AUW'].min() - 100, cessna_data_trimmed_auw['AUW'].max() + 100)
    plt.ylim(filtered_df[param].min() - 5, filtered_df[param].max() + 5)

    # Add titles and labels
    plt.title(f'{param} vs. AUW for Cessna and Piper Aircraft')
    plt.xlabel('AUW (lbs)')
    plt.ylabel(param)
    plt.legend()
    plt.grid()

plt.tight_layout()
plt.show()

# Compare regression coefficients for each parameter
for param in parameters:
    for company in ['Cessna Aircraft Company', 'Piper Aircraft']:
        if company == 'Cessna Aircraft Company':
            X_wing_Span = cessna_data_trimmed['Wing_Span'].values.reshape(-1, 1)
            y_param = cessna_data_trimmed[param].values
        else:
            X_wing_Span = piper_data['Wing_Span'].values.reshape(-1, 1)
            y_param = piper_data[param].values

        model_wing_Span = LinearRegression().fit(X_wing_Span, y_param)
        print(f"{company} ({param} vs. Wing_Span) - Coefficient: {model_wing_Span.coef_[0]:.2f}, Intercept: {model_wing_Span.intercept_:.2f}")

        if company == 'Cessna Aircraft Company':
            X_auw = cessna_data_trimmed['AUW'].values.reshape(-1, 1)
        else:
            X_auw = piper_data['AUW'].values.reshape(-1, 1)

        model_auw = LinearRegression().fit(X_auw, y_param)
        print(f"{company} ({param} vs. AUW) - Coefficient: {model_auw.coef_[0]:.2f}, Intercept: {model_auw.intercept_:.2f}")


---

## Range dependencie on proportion of fuel Weight

Please see the Range vs Fuel weight in the A section for an explenation on how these two features are related to each other.


With outlier removed!

Surprising and interesting result that the regression line turns negative in the upper half of the range. The few number of datapoints suggest that this might not be an accurate reflection of the data but rather stem from a misstake by the regression method similar to that of "Simpson's Paradox". We also see the confidence interval (shadowy area around the regression line) being very large for these datapoints.

In [ ]:
# Calculate Wi / Wf based on the formula
df['Wi/Wf'] = (df['AUW'] + df['FW']) / df['AUW']

# Identify and remove the outlier
removed_outlier = df[df['Wi/Wf'] > 5]
df_filtered = df[df['Wi/Wf'] <= 5]  # DataFrame without the outlier

# Print the removed outlier
print("Removed Outlier:")
print(removed_outlier[['Model', 'Wi/Wf', 'Range']])

# Create a figure for the regression plots
plt.figure(figsize=(10, 6))

# Define intervals for Wi/Wf
interval_1 = df_filtered[df_filtered['Wi/Wf'] <= 1.35]
interval_2 = df_filtered[df_filtered['Wi/Wf'] > 1.35]

# Plot regression for Piper and Cessna in Interval 1
if not interval_1[interval_1['Company'].str.contains('Piper', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_1[interval_1['Company'].str.contains('Piper', case=False)],
                label='Piper (Wi/Wf ≤ 1.35)', color='blue')

if not interval_1[interval_1['Company'].str.contains('Cessna', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_1[interval_1['Company'].str.contains('Cessna', case=False)],
                label='Cessna (Wi/Wf ≤ 1.35)', color='orange')

# Repeat for Interval 2
if not interval_2[interval_2['Company'].str.contains('Piper', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_2[interval_2['Company'].str.contains('Piper', case=False)],
                label='Piper (Wi/Wf > 1.35)', color='cyan', line_kws={'linestyle': '--'})

if not interval_2[interval_2['Company'].str.contains('Cessna', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_2[interval_2['Company'].str.contains('Cessna', case=False)],
                label='Cessna (Wi/Wf > 1.35)', color='red', line_kws={'linestyle': '--'})

# Automatically set axis limits based on data
plt.autoscale()  # This will automatically adjust the limits
plt.gca().set_aspect('auto')  # Set the aspect ratio to be auto-adjusted

# Customize the plot
plt.title('Regression Plot of Range vs. Wi/Wf for Piper and Cessna (Outlier Removed)')
plt.xlabel('Wi/Wf')
plt.ylabel('Range')
plt.grid()
plt.legend()
plt.show()


In [ ]:
print(df[['Wi/Wf', 'Range']].describe())

---

In [ ]:
# Create a figure for the regression plots
plt.figure(figsize=(10, 6))

# Define intervals for Wi/Wf
interval_1 = df[df['Wi/Wf'] <= 1.35]
interval_2 = df[df['Wi/Wf'] > 1.35]

# Plot regression for Piper and Cessna in Interval 1
if not interval_1[interval_1['Company'].str.contains('Piper', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_1[interval_1['Company'].str.contains('Piper', case=False)],
                label='Piper (Wi/Wf ≤ 1.35)', color='blue')

if not interval_1[interval_1['Company'].str.contains('Cessna', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_1[interval_1['Company'].str.contains('Cessna', case=False)],
                label='Cessna (Wi/Wf ≤ 1.35)', color='orange')

# Repeat for Interval 2
if not interval_2[interval_2['Company'].str.contains('Piper', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_2[interval_2['Company'].str.contains('Piper', case=False)],
                label='Piper (Wi/Wf > 1.35)', color='cyan', line_kws={'linestyle': '--'})

if not interval_2[interval_2['Company'].str.contains('Cessna', case=False)].empty:
    sns.regplot(x='Wi/Wf', y='Range', data=interval_2[interval_2['Company'].str.contains('Cessna', case=False)],
                label='Cessna (Wi/Wf > 1.35)', color='red', line_kws={'linestyle': '--'})

# Automatically set axis limits based on data
plt.autoscale()  # This will automatically adjust the limits
plt.gca().set_aspect('auto')  # Set the aspect ratio to be auto-adjusted

# Customize the plot
plt.title('Regression Plot of Range vs. Wi/Wf for Piper and Cessna')
plt.xlabel('Wi/Wf')
plt.ylabel('Range')
plt.grid()
plt.legend()
plt.show()


And the same plot with interactivity (ignore the divide by zero complaint as this will be solved in the streamlit implementation)

In [ ]:
# Create a new DataFrame for hover data
df['Wi/Wf'] = (df['AUW'] + df['FW']) / df['AUW']

# Define intervals for Wi/Wf
interval_1 = df[df['Wi/Wf'] <= 1.35]
interval_2 = df[df['Wi/Wf'] > 1.35]

# Create a plotly scatter plot
fig = px.scatter(df, x='Wi/Wf', y='Range', color='Company',
                 hover_name='Model',  # Column to display on hover
                 title='Scatter Plot of Range vs. Wi/Wf for Piper and Cessna',
                 labels={'Wi/Wf': 'Wi/Wf', 'Range': 'Range'},
                 range_x=[df['Wi/Wf'].min() - 0.1, df['Wi/Wf'].max() + 0.1],
                 range_y=[df['Range'].min() - 100, df['Range'].max() + 100])

# Add regression lines using plotly's `trendline` feature
for company in df['Company'].unique():
    company_data = df[df['Company'] == company]
    if not company_data.empty:
        fig.add_traces(px.scatter(company_data, x='Wi/Wf', y='Range', trendline='ols',
                                  title=f"{company} Regression Line").data)

# Show the figure
fig.show()
